## Preparation

In [1]:
%%capture
!conda install -c pykaldi pykaldi -y

In [2]:
%cd /tmp

/tmp


In [3]:
!git clone https://github.com/jimregan/fairseq/ --branch issue3581

Cloning into 'fairseq'...


remote: Enumerating objects: 28296, done.
remote: Counting objects: 100% (708/708), done.


remote: Compressing objects: 100% (401/401), done.


remote: Total 28296 (delta 389), reused 557 (delta 291), pack-reused 27588
Receiving objects: 100% (28296/28296), 11.91 MiB | 16.69 MiB/s, done.


Resolving deltas: 100% (21212/21212), done.


In [4]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...


remote: Enumerating objects: 13824, done.
remote: Counting objects: 100% (137/137), done.


remote: Compressing objects: 100% (79/79), done.


remote: Total 13824 (delta 76), reused 92 (delta 45), pack-reused 13687
Receiving objects: 100% (13824/13824), 5.49 MiB | 11.43 MiB/s, done.


Resolving deltas: 100% (7956/7956), done.


In [5]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [6]:

%%capture
%cd kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4
%cd /tmp

In [7]:
%%capture
%cd /tmp/kenlm
!python setup.py install
%cd /tmp

In [8]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/tmp/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [9]:
%cd /tmp/fairseq/

/tmp/fairseq


In [10]:
%%capture
!python setup.py install

In [11]:
%cd /tmp/fairseq/

/tmp/fairseq


In [12]:
os.environ['HYDRA_FULL_ERROR'] = '1'

In [13]:
%%capture
!pip install editdistance

## GAN

In [14]:
%%writefile rungan.sh
PREFIX=w2v_unsup_gan_xp
#TASK_DATA=/path/to/features/unfiltered/precompute_unfiltered_pca512_cls128_mean_pooled
TASK_DATA=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/
#TEXT_DATA=/path/to/data  # path to fairseq-preprocessed GAN data
TEXT_DATA=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/
#KENLM_PATH=/path/to/data/kenlm.phn.o4.bin  # KenLM 4-gram phoneme language model (LM data = GAN data here)
KENLM_PATH=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin

PREFIX=$PREFIX fairseq-hydra-train \
	-m --config-dir fairseq/config/model/wav2vecu/gan \
	--config-name w2vu \
	task.data=${TASK_DATA} \
	task.text_data=${TEXT_DATA} \
	task.kenlm_path=${KENLM_PATH} \
	checkpoint.no_epoch_checkpoints=false \
	checkpoint.keep_last_epochs=20 \
	checkpoint.save_dir=/kaggle/working \
	'common.seed=range(0,5)'

Writing rungan.sh


In [15]:
!bash rungan.sh

[2021-06-01 10:06:56,087][HYDRA] Launching 5 jobs locally
[2021-06-01 10:06:56,087][HYDRA] 	#0 : task.data=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/ task.text_data=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/ task.kenlm_path=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin checkpoint.no_epoch_checkpoints=False checkpoint.keep_last_epochs=20 checkpoint.save_dir=/kaggle/working common.seed=0


[2021-06-01 10:06:58,346][fairseq_cli.train][INFO] - {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': 'json', 'log_file': None, 'tensorboard_logdir': 'tb', 'wandb_project': None, 'azureml_logging': False, 'seed': 0, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': True, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'path': None, 'post_process': None, 'quiet': False, 'model_overri

[2021-06-01 10:06:58,478][fairseq_cli.train][INFO] - Wav2vec_U(
  (discriminator): Discriminator(
    (net): Sequential(
      (0): Conv1d(45, 384, kernel_size=(6,), stride=(1,), padding=(5,))
      (1): SamePad()
      (2): Dropout(p=0.0, inplace=False)
      (3): Sequential(
        (0): Conv1d(384, 384, kernel_size=(6,), stride=(1,), padding=(5,))
        (1): SamePad()
        (2): Dropout(p=0.0, inplace=False)
        (3): GELU()
      )
      (4): Conv1d(384, 1, kernel_size=(6,), stride=(1,), padding=(5,))
      (5): SamePad()
    )
  )
  (segmenter): JoinSegmenter()
  (generator): Generator(
    (dropout): Dropout(p=0.1, inplace=False)
    (proj): Sequential(
      (0): TransposeLast()
      (1): Conv1d(512, 45, kernel_size=(4,), stride=(1,), padding=(2,), bias=False)
      (2): TransposeLast()
    )
  )
)
[2021-06-01 10:06:58,479][fairseq_cli.train][INFO] - task: UnpairedAudioText
[2021-06-01 10:06:58,479][fairseq_cli.train][INFO] - model: Wav2vec_U
[2021-06-01 10:06:58,479][fa

[2021-06-01 10:06:58,587][fairseq.data.extracted_features_dataset][INFO] - loaded 2019, skipped 0 samples
[2021-06-01 10:06:58,587][fairseq.tasks.unpaired_audio_text][INFO] - split valid has unpaired text? False
[2021-06-01 10:06:58,596][fairseq_cli.train][INFO] - training on 1 devices (GPUs/TPUs)
[2021-06-01 10:06:58,596][fairseq_cli.train][INFO] - max tokens per device = None and max sentences per device = 160
[2021-06-01 10:06:58,598][fairseq.trainer][INFO] - Preparing to load checkpoint /kaggle/working/checkpoint_last.pt
[2021-06-01 10:06:58,598][fairseq.trainer][INFO] - No existing checkpoint found /kaggle/working/checkpoint_last.pt
[2021-06-01 10:06:58,598][fairseq.trainer][INFO] - loading train data for epoch 1


[2021-06-01 10:06:58,668][fairseq.data.extracted_features_dataset][INFO] - loaded 2331, skipped 0 samples
[2021-06-01 10:06:58,669][fairseq.tasks.unpaired_audio_text][INFO] - split train has unpaired text? True


...

[2021-06-01 18:57:52,192][fairseq.tasks.unpaired_audio_text][INFO] - REF: ɛ n f œ ʂ ə n a d ɵ ʂ ə k t f øː r d eː t s ɔ m h ɛ n d ə p oː ɕ œ r k ɔ n s ɛ t ə n
[2021-06-01 18:57:52,196][fairseq.tasks.unpaired_audio_text][INFO] - HYP: ɕ ɪ ɕ œ b oː ʂ oː ʃ œ d ə n j iː v ə n ə h ɔ b ə r s eː r v iː r l ɪ l k ɪ l s h œ tː a tː t
[2021-06-01 18:57:52,200][fairseq.tasks.unpaired_audio_text][INFO] - LM [REF]: -53.44462585449219, 0.05339602260269112
[2021-06-01 18:57:52,200][fairseq.tasks.unpaired_audio_text][INFO] - LM [HYP]: -79.57679748535156, 0.015539477815292604


[2021-06-01 18:57:52,713][valid][INFO] - {"epoch": 702, "valid_loss": "0.988", "valid_ntokens": "3039.79", "valid_nsentences": "144.214", "valid_lm_score_sum": "-87945.7", "valid_num_pred_chars": "47152", "valid_vocab_seen_pct": "0.898955", "valid_uer": "98.7546", "valid_weighted_lm_ppl": "76.049", "valid_lm_ppl": "61.4567", "valid_wps": "16102.5", "valid_wpb": "3039.8", "valid_bsz": "144.2", "valid_num_updates": "11232", "valid_best_weighted_lm_ppl": "73.9191"}
[2021-06-01 18:57:52,717][fairseq.checkpoint_utils][INFO] - Preparing to save checkpoint for epoch 702 @ 11232 updates
[2021-06-01 18:57:52,719][fairseq.trainer][INFO] - Saving checkpoint to /kaggle/working/checkpoint702.pt


[2021-06-01 18:57:52,754][fairseq.trainer][INFO] - Finished saving checkpoint to /kaggle/working/checkpoint702.pt
[2021-06-01 18:57:52,786][fairseq.checkpoint_utils][INFO] - Saved checkpoint /kaggle/working/checkpoint702.pt (epoch 702 @ 11232 updates, score 76.04904870654075) (writing took 0.06901522199768806 seconds)
[2021-06-01 18:57:52,789][fairseq_cli.train][INFO] - end of epoch 702 (average epoch stats below)
[2021-06-01 18:57:52,792][train][INFO] - {"epoch": 702, "train_loss": "2.51", "train_ntokens": "145.688", "train_nsentences": "145.688", "train_temp": "1.141", "train_code_ppl": "8.319", "train_loss_code_pen": "0.241", "train_loss_smoothness": "1.664", "train_loss_dense_g": "2.341", "train_lm_score_sum": 0.0, "train_num_pred_chars": 0.0, "train_loss_grad_pen": "0.162", "train_loss_dense_d": "0.127", "train_loss_token_d": "0.106", "train_wps": "45.9", "train_ups": "0.32", "train_wpb": "145.7", "train_bsz": "145.7", "train_num_updates": "11232", "train_lr_discriminator": "0.000

[2021-06-01 18:57:52,849][fairseq.trainer][INFO] - begin training epoch 703
[2021-06-01 18:57:52,852][fairseq_cli.train][INFO] - Start iterating over samples


[2021-06-01 18:58:40,700][fairseq_cli.train][INFO] - begin validation on "valid" subset
